In [1]:
import numpy as np
import math
import os
import time
import logging
from hyperopt.plotting import *
from hyperopt import fmin, tpe, hp, STATUS_OK, STATUS_FAIL, Trials, base, rand
from data.data_loader import Dataset
from misc.preferences import PREFERENCES
from misc.run_configuration import from_hyperopt, OutputLayerType, LearningSchedulerType, OptimizerType, default_params
from misc import utils
from misc.hyperopt_space import *

from optimizer import get_optimizer
from criterion import NllLoss, LossCombiner
from models.transformer.encoder import TransformerEncoder
from models.jointAspectTagger import JointAspectTagger
from trainer.train import Trainer
import pprint
import argparse
import pickle

In [2]:
POSSIBLE_DATASET_VALUES = ['germeval', 'organic', 'amazon']


def load_model(dataset, rc, experiment_name):
    loss = LossCombiner(4, dataset.class_weights, NllLoss)
    transformer = TransformerEncoder(dataset.source_embedding,
                                     hyperparameters=rc)
    model = JointAspectTagger(transformer, rc, 4, 20, dataset.target_names)
    optimizer = get_optimizer(model, rc)
    trainer = Trainer(
                        model,
                        loss,
                        optimizer,
                        rc,
                        dataset,
                        experiment_name,
                        enable_tensorboard=False,
                        verbose=False)
    return trainer

def load_dataset(rc, logger, task):
    dataset = Dataset(
            task,
            logger,
            rc,
            source_index=PREFERENCES.source_index,
            target_vocab_index=PREFERENCES.target_vocab_index,
            data_path=PREFERENCES.data_root,
            train_file=PREFERENCES.data_train,
            valid_file=PREFERENCES.data_validation,
            test_file=PREFERENCES.data_test,
            file_format=PREFERENCES.file_format,
            init_token=None,
            eos_token=None
        )
    dataset.load_data(dsl, verbose=False)
    return dataset

def objective(parameters):
    run_time = time.time()

    utils.reset_loggers()
    experiment_name = utils.create_loggers(experiment_name=main_experiment_name)
    logger = logging.getLogger(__name__)
    dataset_logger = logging.getLogger('data_loader')

    # generate hp's from parameters
    try:
        rc = from_hyperopt(parameters, use_cuda, model_size=300, early_stopping=5, num_epochs=35, log_every_xth_iteration=-1, language=PREFERENCES.language)
    except Exception as err:
        print('Could not convert params: ' + str(err))
        logger.exception("Could not load parameters from hyperopt configuration: " + parameters)
        return {
            'status': STATUS_FAIL,
            'eval_time': time.time() - run_time
        }
    logger.info('New Params:')
    logger.info(rc)
    print('\n\n#########################################################################')
    print(rc)

    logger.debug('Load dataset')
    try:
        dataset = load_dataset(rc, dataset_logger, rc.task)
    except Exception as err:
        print('Could not load dataset: ' + str(err))
        logger.exception("Could not load dataset")
        return {
            'status': STATUS_FAIL,
            'eval_time': time.time() - run_time
        }
    logger.debug('dataset loaded')
    logger.debug('Load model')

    try:
        trainer = load_model(dataset, rc, experiment_name)
    except Exception as err:
        print('Could not load model: ' + str(err))
        logger.exception("Could not load model")
        return {
            'status': STATUS_FAIL,
            'eval_time': time.time() - run_time
        }

    logger.debug('model loaded')

    logger.debug('Begin training')
    model = None
    try:
        result = trainer.train(use_cuda=rc.use_cuda, perform_evaluation=False)
        model = result['model']
    except Exception as err:
        print('Exception while training: ' + str(err))
        logger.exception("Could not complete iteration")
        return {
            'status': STATUS_FAIL,
            'eval_time': time.time() - run_time,
            'best_loss': trainer.get_best_loss(),
            'best_f1': trainer.get_best_f1()
        }

    if math.isnan(trainer.get_best_loss()):
        print('Loss is nan')
        return {
            'status': STATUS_FAIL,
            'eval_time': time.time() - run_time,
            'best_loss': trainer.get_best_loss(),
            'best_f1': trainer.get_best_f1()
        }

    # perform evaluation and log results
    result = None
    try:
        result = trainer.perform_final_evaluation(use_test_set=True, verbose=False)
    except Exception as err:
        logger.exception("Could not complete iteration evaluation.")
        print('Could not complete iteration evaluation: ' + str(err))
        return {
            'status': STATUS_FAIL,
            'eval_time': time.time() - run_time,
            'best_loss': trainer.get_best_loss(),
            'best_f1': trainer.get_best_f1()
        }
    print(f'VAL f1\t{trainer.get_best_f1()} - ({result[1][1]})')
    print(f'VAL loss\t{trainer.get_best_loss()}')

    print(f"       .---.\n \
         /     \\\n\
          \\.@-@./\n\
          /`\\_/`\\\n\
         //  _  \\\\\tLoss: {trainer.get_best_loss()}\n\
        | \\     )|_\tf1: {trainer.get_best_f1()}\n\
       /`\\_`>  <_/ \\\n\
       \\__/'---'\\__/\n")

    return {
            'loss': result[1][0],
            'status': STATUS_OK,
            'eval_time': time.time() - run_time,
            'best_loss': trainer.get_best_loss(),
            'best_f1': trainer.get_best_f1(),
            'sample_iterations': trainer.get_num_samples_seen(),
            'iterations': trainer.get_num_iterations(),
            'rc': rc,
            'results': {
                'train': {
                    'loss': result[0][0],
                    'f1': result[0][1]
                },
                'validation': {
                    'loss': result[1][0],
                    'f1': result[1][1]
                },
                'test': {
                    'loss': result[2][0],
                    'f1': result[2][1]
                }
            }
        }

In [3]:
dataset_choice = 'germeval'
runs = 100
main_experiment_name = 'GermEvalHyperoptRandom'
use_cuda = True
description = 'GermEval random Hyperopt run'

if dataset_choice not in POSSIBLE_DATASET_VALUES:
    raise Error()

In [4]:
if dataset_choice == POSSIBLE_DATASET_VALUES[0]:
    PREFERENCES.defaults(
        data_root='./data/data/germeval2017',
        data_train='train_v1.4.tsv',    
        data_validation='dev_v1.4.tsv',
        data_test='test_TIMESTAMP1.tsv',
        source_index=0,
        target_vocab_index=2,
        file_format='csv',
        language='de'
    )
    from data.germeval2017 import germeval2017_dataset as dsl

    search_space = {
        'batch_size': hp.quniform('batch_size', 10, 100, 1),
        'num_encoder_blocks': hp.quniform('num_encoder_blocks', 1, 8, 1),
        'pointwise_layer_size': hp.quniform('pointwise_layer_size', 32, 256, 1),
        'clip_comments_to': hp.quniform('clip_comments_to', 10, 250, 1),
        'dropout_rate': hp.uniform('dropout_rate', 0.0, 0.8),
        'output_dropout_rate': hp.uniform('last_layer_dropout', 0.0, 0.8),
        'num_heads': hp.choice('num_heads', [1, 2, 3, 4, 5]),
        'transformer_use_bias': hp_bool('transformer_use_bias'),
        'output_layer': hp.choice('output_layer', [
            {
                'type': OutputLayerType.Convolutions,
                'output_conv_num_filters': hp.quniform('output_conv_num_filters', 1, 400, 1),
                'output_conv_kernel_size': hp.quniform('output_conv_kernel_size', 1, 10, 1),
                'output_conv_stride': hp.quniform('output_conv_stride', 1, 10, 1),
                'output_conv_padding': hp.quniform('output_conv_padding', 0, 5, 1),
            },
            {
                'type': OutputLayerType.LinearSum
            }
        ]),
        'learning_rate_scheduler': hp.choice('learning_rate_scheduler', [
            {
                'type': LearningSchedulerType.Noam,
                'noam_learning_rate_warmup': hp.quniform('noam_learning_rate_warmup', 1000, 9000, 1),
                'noam_learning_rate_factor': hp.uniform('noam_learning_rate_factor', 0.01, 4)
            }
        ]),
        'optimizer': hp.choice('optimizer', [
            {
                'type': OptimizerType.Adam,
                'adam_beta1': hp.uniform('adam_beta1', 0.7, 0.999),
                'adam_beta2': hp.uniform('adam_beta2', 0.7, 0.999),
                'adam_eps': hp.loguniform('adam_eps', np.log(1e-10), np.log(1)),
                'learning_rate': hp.lognormal('adam_learning_rate', np.log(0.01), np.log(10)),
                'adam_weight_decay': 1*10**hp.quniform('adam_weight_decay', -8, -3, 1)
            },
            #{
            #    'type': OptimizerType.SGD,
            #    'sgd_momentum': hp.uniform('sgd_momentum', 0.4, 1),
            #    'sgd_weight_decay': hp.loguniform('sgd_weight_decay', np.log(1e-4), np.log(1)),
            #    'sgd_nesterov': hp_bool('sgd_nesterov'),
            #    'learning_rate': hp.lognormal('sgd_learning_rate', np.log(0.01), np.log(10))
        ]),
        'replace_url_tokens': hp_bool('replace_url_tokens'),
        'harmonize_bahn': hp_bool('harmonize_bahn'),
        'embedding_type': hp.choice('embedding_type', ['fasttext', 'glove']),
        'embedding_name': hp.choice('embedding_name', ['6B']),
        'embedding_dim': hp.choice('embedding_dim', [300]),
        'use_stop_words': hp_bool('use_stop_words'),
        'use_spell_checker': hp_bool('use_spell_checker'),
        'embedding_type': hp.choice('embedding_type', ['fasttext', 'glove']),
        'task': 'germeval'
    }

elif dataset_choice == POSSIBLE_DATASET_VALUES[1]:
     from data.organic2019 import organic_dataset as dsl
     from data.organic2019 import ORGANIC_TASK_ALL, ORGANIC_TASK_ENTITIES, ORGANIC_TASK_ATTRIBUTES, ORGANIC_TASK_ENTITIES_COMBINE, ORGANIC_TASK_COARSE
     PREFERENCES.defaults(
        data_root='./data/data/organic2019',
        data_train='train.csv',    
        data_validation='validation.csv',
        data_test='test.csv',
        source_index=0,
        target_vocab_index=1,
        file_format='csv',
        language='en'
     )

     search_space = {
        'batch_size': hp.quniform('batch_size', 10, 64, 1),
        'num_encoder_blocks': hp.quniform('num_encoder_blocks', 1, 4, 1),
        'pointwise_layer_size': hp.quniform('pointwise_layer_size', 32, 350, 1),
        'clip_comments_to': hp.quniform('clip_comments_to', 45, 180, 1),
        'dropout_rate': hp.uniform('dropout_rate', 0.0, 0.8),
        'output_dropout_rate': hp.uniform('last_layer_dropout', 0.0, 0.8),
        'num_heads': hp.choice('num_heads', [1, 2, 3, 4, 5]),
        'transformer_use_bias': hp_bool('transformer_use_bias'),
        'output_layer': hp.choice('output_layer', [
            {
                'type': OutputLayerType.Convolutions,
                'output_conv_num_filters': hp.quniform('output_conv_num_filters', 10, 400, 1),
                'output_conv_kernel_size': hp.quniform('output_conv_kernel_size', 1, 10, 1),
                'output_conv_stride': hp.quniform('output_conv_stride', 1, 10, 1),
                'output_conv_padding': hp.quniform('output_conv_padding', 0, 5, 1),
            },
            {
                'type': OutputLayerType.LinearSum
            }
        ]),
        'learning_rate_scheduler': hp.choice('learning_rate_scheduler', [
            {
                'type': LearningSchedulerType.Noam,
                'noam_learning_rate_warmup': hp.quniform('noam_learning_rate_warmup', 1000, 9000, 1),
                'noam_learning_rate_factor': hp.uniform('noam_learning_rate_factor', 0.01, 4)
            }
        ]),
        'optimizer': hp.choice('optimizer', [
            {
                'type': OptimizerType.Adam,
                'adam_beta1': hp.uniform('adam_beta1', 0.7, 0.999),
                'adam_beta2': hp.uniform('adam_beta2', 0.7, 0.999),
                'adam_eps': hp.loguniform('adam_eps', np.log(1e-10), np.log(1)),
                'learning_rate': hp.lognormal('adam_learning_rate', np.log(0.01), np.log(10)),
                'adam_weight_decay': 1*10**hp.quniform('adam_weight_decay', -8, -3, 1)
            },
            #{
            #    'type': OptimizerType.SGD,
            #    'sgd_momentum': hp.uniform('sgd_momentum', 0.4, 1),
            #    'sgd_weight_decay': hp.loguniform('sgd_weight_decay', np.log(1e-4), np.log(1)),
            #    'sgd_nesterov': hp_bool('sgd_nesterov'),
            #    'learning_rate': hp.lognormal('sgd_learning_rate', np.log(0.01), np.log(10))
        ]),
        'task': hp.choice('task', [
            ORGANIC_TASK_ENTITIES,
            ORGANIC_TASK_ENTITIES_COMBINE
        ]),
        'use_stop_words': hp_bool('use_stop_words'),
        'use_spell_checker': hp_bool('use_spell_checker'),
        'embedding_type': hp.choice('embedding_type', ['fasttext', 'glove'])
    }
else:
    PREFERENCES.defaults(
        data_root='./data/data/amazon/splits',
        data_train='train.pkl',    
        data_validation='val.pkl',
        data_test='test.pkl',
        source_index=0,
        target_vocab_index=1,
        file_format='pkl',
        language='en'
    )
    from data.amazon import amazon_dataset as dsl

In [5]:
experiment_name = utils.create_loggers(experiment_name=main_experiment_name)
logger = logging.getLogger(__name__)
dataset_logger = logging.getLogger('data_loader')
logger.info('Run hyper parameter random grid search for experiment with name ' + main_experiment_name)
logger.info('num_optim_iterations: ' + str(runs))

Log path is  C:\Users\felix\OneDrive\Studium\Studium\6. Semester\MA\Project\ABSA-Transformer\logs\GermEvalHyperoptRandom\20190424\0


In [ ]:
try:
    logger.info('Current commit: ' + utils.get_current_git_commit())
    print('Current commit: ' + utils.get_current_git_commit())
except Exception as err:
    logger.exception('Could not print current commit')

trials = Trials()
try:

    best = fmin(objective,
        space=search_space,
        algo=rand.suggest,
        max_evals=runs,
        trials=trials)

    print(best)
except Exception as err:
    logger.exception('Could not complete optimization')
    print('Could not complete optimization. The log file provides more details.')


path = os.path.join(os.getcwd(), 'logs', f'hp_run_{main_experiment_name}.pkl')
with open(path, 'wb') as f:
    pickle.dump(trials, f)

Current commit: b'57a7aff'
Log path is                                                                                                            
C:\Users\felix\OneDrive\Studium\Studium\6. Semester\MA\Project\ABSA-Transformer\logs\GermEvalHyperoptRandom\20190424\1 
                                                                                                                       

#########################################################################
+----------------------------------------------------------------------------------+                                   
|                                 Hyperparameters                                  |
+------------------------------+---------------------------------------------------+
|          Parameter           |                       Value                       |
+------------------------------+---------------------------------------------------+
|            kwargs            | {'batch_size': 66.0, 'learning_rate_sche[...]

A Jupyter Widget

# EP	# IT	tr loss		val loss	f1		acc		duration / total time                                                             
1	17k	0.30		0.22		0.234		0.896		0.84m - 0.8m / 0.0m                                                                    
2	34k	0.23		0.20		0.272		0.912		0.83m - 1.7m / 29.5m                                                                   
3	51k	0.21		0.20		0.211		0.863		0.78m - 2.5m / 29.1m                                                                   
4	68k	0.20		0.19		0.199		0.848		0.84m - 3.4m / 27.4m                                                                   
5	85k	0.18		0.19		0.256		0.894		0.82m - 4.2m / 29.3m                                                                   
6	103k	0.17		0.20		0.217		0.870		0.83m - 5.0m / 28.8m                                                                  
7	120k	0.15		0.22		0.331		0.929		0.80m - 5.8m / 29.1m                                                                  
8	137k	0.14		0.22		0.220		0.869		0.82m -

A Jupyter Widget

Exception while training: size mismatch, m1: [12956 x 3], m2: [164 x 4] at c:\a\w\1\s\tmp_conda_3.6_105809\conda\conda-bld\pytorch_1544094150554\work\aten\src\thc\generic/THCTensorMathBlas.cu:266
Log path is                                                                                                            
C:\Users\felix\OneDrive\Studium\Studium\6. Semester\MA\Project\ABSA-Transformer\logs\GermEvalHyperoptRandom\20190424\3 
                                                                                                                       

#########################################################################
+----------------------------------------------------------------------------------+                                   
|                                 Hyperparameters                                  |
+------------------------------+---------------------------------------------------+
|          Parameter           |                       Value                   

...

# EP	# IT	tr loss		val loss	f1		acc		duration / total time                                                             
1	17k	0.80		0.77		0.158		0.826		2.39m - 2.4m / 0.0m                                                                    
2	34k	0.63		0.84		0.280		0.913		2.44m - 4.9m / 83.5m                                                                   
3	51k	0.63		0.88		0.281		0.917		2.41m - 7.3m / 85.5m                                                                   
4	68k	0.64		1.17		0.288		0.942		2.49m - 9.8m / 84.6m                                                                   
5	85k	0.64		1.12		0.322		0.933		2.40m - 12.2m / 87.1m                                                                  
6	102k	0.65		1.22		0.339		0.948		2.42m - 14.7m / 84.1m                                                                 
7	119k	0.63		1.66		0.346		0.952		2.38m - 17.1m / 84.9m                                                                 
8	136k	0.62		2.05		0.388		0.959		2.40m -

A Jupyter Widget

# EP	# IT	tr loss		val loss	f1		acc		duration / total time                                                             
1	17k	0.45		0.33		0.023		0.852		2.65m - 2.6m / 0.0m                                                                    
2	34k	0.29		0.29		0.382		0.959		2.63m - 5.3m / 92.7m                                                                   
3	51k	0.26		0.30		0.265		0.916		2.64m - 8.0m / 92.2m                                                                   
4	68k	0.26		0.29		0.265		0.916		2.75m - 10.7m / 92.5m                                                                  
5	85k	0.26		0.29		0.265		0.916		2.68m - 13.4m / 96.0m                                                                  
6	102k	0.25		0.29		0.265		0.916		2.65m - 16.1m / 93.8m                                                                 
7	119k	0.26		0.29		0.265		0.916		2.65m - 18.8m / 92.9m                                                                 
VAL f1	0.38205453392517436 - (0.38205453

A Jupyter Widget

# EP	# IT	tr loss		val loss	f1		acc		duration / total time                                                             
1	17k	0.29		0.28		0.246		0.912		2.54m - 2.5m / 0.0m                                                                    
2	34k	0.30		0.29		0.212		0.874		2.63m - 5.2m / 88.8m                                                                   
3	51k	0.30		0.28		0.265		0.916		2.60m - 7.8m / 91.8m                                                                   
4	68k	0.28		0.26		0.265		0.916		2.88m - 10.7m / 91.1m                                                                  
 78%|████████████████████████████████▊         | 78/100 [28:34:28<8:18:47, 1360.35s/it, best loss: 0.10995660924978375]